In [1]:
import numpy as np, pandas as pd

### Load

In [2]:
df = pd.read_csv('data/train.tsv', sep='\t', quotechar=' ', header = None).fillna('-')
df.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']
test = pd.read_csv('data/public.tsv', sep='\t', quotechar = ' ', header = None).fillna('-')
test.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

In [7]:
df.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,2,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",0,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,2,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,2,0.838054


In [3]:
def label_enc(x ,reverse = False):
    if reverse == False:
        if x == 'bad':
            return 0
        elif x == 'neutral':
            return 1
        else:
            return 2
    else:
        if x == 0:
            return 'bad'
        elif x == 1:
            return 'neutral'
        else:
            return 'good'
df['label'] = df['label'].apply(lambda x: label_enc(x))

In [4]:
from keras.preprocessing.text import Tokenizer, hashing_trick
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

/home/denis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split

import scipy.sparse as sps

In [10]:
df[[col for col in df.columns if col not in ['label', 'confidence']]]

,context_id,context_2,context_1,context_0,reply_id,reply
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я ."
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти ."
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .
5,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",5,просто повесь трубку .
6,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...",0,едете ?
7,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...",1,едем .
8,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...",2,"куда же вы едете , месье ?"
9,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились .","я пытаюсь добраться до юджина , но , кажется ,...",3,"берите бутылку , а мы такси ."


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df[[col for col in df.columns if col not in ['label', 'confidence']]], df['label'], train_size=0.3)

/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:
def batch_generator(X, y, batch_size, shuffle):
    number_of_batches = X.shape[0]/batch_size
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

In [13]:
model = Sequential()
model.add(Embedding(100, 128, input_length=4194304))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
model.fit_generator(generator=batch_generator(X_test, y_test, 32, True),
                                    nb_epoch=3,
                                    samples_per_epoch=X_train.shape[0])

Train...


/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  del sys.path[0]
/home/denis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., steps_per_epoch=29259, epochs=3)`
  del sys.path[0]


Epoch 1/3


TypeError: unhashable type: 'numpy.ndarray'